In [9]:
!pip install google-cloud-aiplatform gradio pandas
!pip install pymupdf pdfplumber python-docx xmltodict lxml beautifulsoup4
!pip install langchain_google_vertexai langchain
!pip install nltk 
!pip install scispacy 


  Using cached scispacy-0.5.5-py3-none-any.whl.metadata (18 kB)
  Using cached spacy-3.7.5.tar.gz (1.3 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [510 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Using cached cymem-2.0.11-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.5 kB)
        Using cached preshed-3.0.10-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.4 kB)
        Using cached murmurhash-1.0.13-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Using cached thinc-8.2.5.tar.gz (193 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'error'
        error: subprocess-ex

In [10]:
# !pip install scispacy
# !pip install https://s3-us-west-2.amazonaws.com/ai2-s2-scispacy/releases/v0.5.1/en_core_sci_sm-0.5.1.tar.gz

!pip install -U spacy==3.7.2
!python -m spacy download en_core_web_sm
!pip install --upgrade pip setuptools wheel
!pip install --upgrade numpy
!pip install --upgrade spacy

!pip install --force-reinstall --no-cache-dir numpy spacy thinc cymem murmurhash preshed blis
!python -m spacy download en_core_web_sm

  Using cached spacy-3.7.2.tar.gz (1.3 MB)
  Installing build dependencies ... error
  error: subprocess-exited-with-error
  
  × pip subprocess to install build dependencies did not run successfully.
  │ exit code: 1
  ╰─> [510 lines of output]
      Ignoring numpy: markers 'python_version < "3.9"' don't match your environment
        Using cached setuptools-80.9.0-py3-none-any.whl.metadata (6.6 kB)
        Using cached Cython-0.29.37-py2.py3-none-any.whl.metadata (3.1 kB)
        Using cached cymem-2.0.11-cp313-cp313-macosx_11_0_arm64.whl.metadata (8.5 kB)
        Using cached preshed-3.0.10-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.4 kB)
        Using cached murmurhash-1.0.13-cp313-cp313-macosx_11_0_arm64.whl.metadata (2.2 kB)
        Using cached thinc-8.2.5.tar.gz (193 kB)
        Installing build dependencies: started
        Installing build dependencies: finished with status 'error'
        error: subprocess-exited-with-error
      
        × pip subprocess to install build

In [11]:
import spacy
nlp = spacy.load("en_core_web_sm")
doc = nlp("The system shall encrypt PHI using AES-256.")
print([token.text for token in doc if token.pos_ == "VERB"])

['encrypt', 'using']


In [ ]:
import nltk
nltk.download("punkt")
nltk.download("punkt_tab")  # newer versions need this


In [2]:
from google.cloud import bigquery
client = bigquery.Client(project="gen-470713")  # or rely on GOOGLE_CLOUD_PROJECT
project_id = client.project
print(project_id)
import os
os.environ["GOOGLE_CLOUD_PROJECT"] = project_id
os.environ["GOOGLE_CLOUD_LOCATION"] = "us-central1"




gen-470713


# Parse Documents

In [3]:
import importlib
import batch_parser
importlib.reload(batch_parser)

from batch_parser import BatchParser

# Step 1: Parse docs
parser = BatchParser(data_folder="data")
parsed_results = parser.parse_batch()

# Step 2: Export to BigQuery
parser.export_results(
    results=parsed_results,
    project_id=project_id,
    dataset_id="requirements_dataset",
    table_id="raw_chunks"
)


# Show parsed results
for fname, chunks in parsed_results.items():
    print(f"\n--- {fname} ---")
    for i, chunk in enumerate(chunks[:5]):  # show first 5 chunks
        print(f"{i+1}. {chunk}")


⚠️ Table gen-470713.requirements_dataset.raw_chunks not found. Creating...
⚠️ BigQuery export failed: 404 POST https://bigquery.googleapis.com/bigquery/v2/projects/gen-470713/datasets/requirements_dataset/tables?prettyPrint=false: Not found: Dataset gen-470713:requirements_dataset
💾 Saved 84 requirements locally → requirements.json

--- data/Common_InsuranceReqs_FINAL.pdf ---
1. {'id': 'c446d027-23db-4c75-9089-4c6595335289', 'requirement_id': 'REQ-001', 'filename': 'Common_InsuranceReqs_FINAL.pdf', 'statement': 'These requirements have since been used by a variety of stakeholders, including the Open Health Information Exchange (OpenHIE), the open source Health Insurance Management Information System (openIMIS), and several country governments, including that of Bangladesh.', 'created_at': '2025-08-31T14:12:13.249578'}
2. {'id': '9b48fb6b-d892-4a5b-99ad-0765edcd941d', 'requirement_id': 'REQ-002', 'filename': 'Common_InsuranceReqs_FINAL.pdf', 'statement': 'These requirements have since b

## Requirement Registry Layer (Functional Step 2)

Purpose: Convert raw requirement candidates into structured requirements.

In [4]:
llm_model="gemini-2.5-pro"
project_id

'gen-470713'

In [23]:
import importlib
import requirement_builder
importlib.reload(requirement_builder)
from requirement_builder import RequirementBuilder
import json

builder = RequirementBuilder(model=llm_model, project_id=project_id)

# ------------------ Load Input from Layer 1 ------------------
# Layer 1 (BatchParser) saved clean requirements into requirements.json
with open("requirements.json", "r", encoding="utf-8") as f:
    requirements = json.load(f)  
    
raw_reqs = requirements[:5]

# ------------------ Build Structured Requirements (Layer 2) ------------------
structured_reqs = builder.build_registry(raw_reqs, batch_size=2)

# Save Layer 2 output
with open("structured_requirements.json", "w", encoding="utf-8") as f:
    json.dump(structured_reqs, f, indent=2)

print(f"💾 Saved {len(structured_reqs)} structured requirements → structured_requirements.json")

# ------------------ Export to BigQuery ------------------
try:
    builder.export_to_bq(structured_reqs, project_id=project_id, dataset_id="requirements_dataset")
except Exception as e:
    print(f"⚠️ BigQuery export failed: {e} (saved locally instead)")


Processing requirements:   0%|          | 0/3 [00:00<?, ?it/s]Retrying langchain_google_vertexai.chat_models._completion_with_retry.<locals>._completion_with_retry_inner in 4.0 seconds as it raised PermissionDenied: 403 This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "BILLING_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gen-470713"
}
metadata {
  key: "consumer"
  value: "projects/gen-470713"
}
, locale: "en-US"
message: "This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enab

PermissionDenied: 403 This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "BILLING_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gen-470713"
}
metadata {
  key: "consumer"
  value: "projects/gen-470713"
}
, locale: "en-US"
message: "This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry."
, links {
  description: "Google developers console billing"
  url: "https://console.

In [11]:
%pip install -U json5

  Using cached json5-0.12.1-py3-none-any.whl.metadata (36 kB)
Using cached json5-0.12.1-py3-none-any.whl (36 kB)
Note: you may need to restart the kernel to use updated packages.


In [15]:
structured_reqs[0]

NameError: name 'structured_reqs' is not defined

# Layer 3 – Metadata Enrichment

Add domain-specific, fine-grained attributes to requirements produced in Layer 2 using rules + NLP, making them richer and standardized for compliance, analytics, and test case mapping.

In [16]:
from langchain_google_vertexai import VertexAI

llm = VertexAI(model_name="gemini-2.5-pro", project=project_id, location="us-central1")


In [18]:
import metadata_enricher
import importlib
importlib.reload(metadata_enricher)
from metadata_enricher import MetadataEnricher
import json

# Load structured requirements from Layer 2
with open("structured_requirements.json", "r", encoding="utf-8") as f:
    structured_reqs = json.load(f)

# Run enrichment
enricher = MetadataEnricher()
enriched_reqs = enricher.enrich(structured_reqs)

# Save output
with open("enriched_requirements.json", "w", encoding="utf-8") as f:
    json.dump(enriched_reqs, f, indent=2)

print(f"✅ Enriched {len(enriched_reqs)} requirements → enriched_requirements.json")


Enriching requirements: 100%|██████████| 5/5 [00:00<00:00, 82.45it/s]

✅ Enriched 5 requirements → enriched_requirements.json


In [24]:
enriched_reqs

[{'requirement_id': 'REQ-001',
  'category': 'Compliance',
  'title': 'Stakeholder and Standards Alignment',
  'statement': 'The system must align with requirements used by key stakeholders, including the Open Health Information Exchange (OpenHIE), the open source Health Insurance Management Information System (openIMIS), and relevant country governments (e.g., Bangladesh), to ensure interoperability and adoption.',
  'priority': 'P1',
  'severity': 'Major',
  'regulation': ['openIMIS', 'OpenHIE'],
  'actors': ['Project Manager', 'Compliance Officer', 'System Architect'],
  'data_type': ['Health Information Exchange Data', 'Health Insurance Data'],
  'action': ['Integrate', 'Align'],
  'acceptance_criteria': ['System design documentation demonstrates mapping and traceability to OpenHIE specifications.',
   'System design documentation demonstrates mapping and traceability to openIMIS specifications, where applicable.',
   "Evidence is provided showing that the system's requirements are

# Layer 4 – Categorization & Retrieval Layer (Functional Step 4)

Purpose: Classify requirements and enable semantic retrieval.

In [20]:
llm_model

'gemini-2.5-pro'

In [21]:
import categorizer_retriever
import importlib
importlib.reload(categorizer_retriever)
from categorizer_retriever import CategorizerRetriever

embedding_model="text-embedding-005"

# Load enriched requirements (Layer 3 output)
with open("enriched_requirements.json", "r", encoding="utf-8") as f:
    enriched_reqs = json.load(f)

cr = CategorizerRetriever(project_id=project_id,embedding_model=embedding_model,classifier_model=llm_model)

# Step 1: Process → categorize + embed
categorized_reqs = cr.process(enriched_reqs)

# Step 2: Save to BigQuery
cr.export_to_bq(categorized_reqs, dataset_id="requirements_dataset", table_id="requirements")


/Users/tanvisaxena/Downloads/ATMGG/venv/lib/python3.13/site-packages/vertexai/_model_garden/_model_garden_models.py:278: UserWarning: This feature is deprecated as of June 24, 2025 and will be removed on June 24, 2026. For details, see https://cloud.google.com/vertex-ai/generative-ai/docs/deprecations/genai-vertexai-sdk.
  warning_logs.show_deprecation_warning()


PermissionDenied: 403 This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry. [reason: "BILLING_DISABLED"
domain: "googleapis.com"
metadata {
  key: "service"
  value: "aiplatform.googleapis.com"
}
metadata {
  key: "containerInfo"
  value: "gen-470713"
}
metadata {
  key: "consumer"
  value: "projects/gen-470713"
}
, locale: "en-US"
message: "This API method requires billing to be enabled. Please enable billing on project #gen-470713 by visiting https://console.developers.google.com/billing/enable?project=gen-470713 then retry. If you enabled billing for this project recently, wait a few minutes for the action to propagate to our systems and retry."
, links {
  description: "Google developers console billing"
  url: "https://console.

In [19]:
categorized_reqs[0]

{'requirement_id': 'REQ-001',
 'category': 'Compliance',
 'title': 'Stakeholder and Standards Alignment',
 'statement': 'The system must align with requirements used by key stakeholders, including the Open Health Information Exchange (OpenHIE), the open source Health Insurance Management Information System (openIMIS), and relevant country governments (e.g., Bangladesh), to ensure interoperability and adoption.',
 'priority': 'P1',
 'severity': 'Major',
 'regulation': ['openIMIS', 'OpenHIE'],
 'actors': ['Project Manager', 'Compliance Officer', 'System Architect'],
 'data_type': ['Health Insurance Data', 'Health Information Exchange Data'],
 'action': ['Integrate', 'Align'],
 'acceptance_criteria': ['System design documentation demonstrates mapping and traceability to OpenHIE specifications.',
  'System design documentation demonstrates mapping and traceability to openIMIS specifications, where applicable.',
  "Evidence is provided showing that the system's requirements are in alignment

# Layer 5 – Test Case Generation Layer 

This layer will:

-- Take requirements from Layer 4 (categorized_reqs).
- Generate 3 test cases per requirement (Positive, Negative, Edge).
- Normalize into a consistent schema.
- Export into BigQuery (test_cases table), just like we did for requirements.

In [26]:
import importlib
import test_case_generator
importlib.reload(test_case_generator)
from test_case_generator import TestCaseGenerator
from google.cloud import bigquery
import json

project_id = project_id
dataset_id = "requirements_dataset"
requirements_table = "requirements"
testcases_table = "test_cases"

# Step 1: Load requirements from BigQuery
client = bigquery.Client(project=project_id)
query = f"""
    SELECT requirement_id, category, title, statement, priority, severity
    FROM `{project_id}.{dataset_id}.{requirements_table}`
"""
reqs = [dict(row) for row in client.query(query).result()]
print(f"📥 Loaded {len(reqs)} requirements from BigQuery")

# ⚡ Take only the first requirement for testing
reqs = reqs[:1]

# Step 2: Generate test cases
tcg = TestCaseGenerator(project_id)
test_cases = tcg.batch_generate(reqs)

# Step 3: Export test cases back into BigQuery
tcg.export_to_bq(test_cases, dataset_id=dataset_id, table_id=testcases_table)

# Optional: save locally too
with open("test_cases.json", "w", encoding="utf-8") as f:
    json.dump(test_cases, f, indent=2)
print("💾 Saved test cases locally → test_cases.json")


NotFound: 404 Not found: Dataset gen-470713:requirements_dataset was not found in location US; reason: notFound, message: Not found: Dataset gen-470713:requirements_dataset was not found in location US

Location: US
Job ID: b6436842-eae2-41d9-8dca-46bbeb0a2107


In [33]:
test_cases

[{'test_id': 'TC_INTEROP_001',
  'requirement_id': 'REQ-001',
  'title': '[Positive] Verify successful data exchange using a standard OpenHIE profile',
  'description': 'This test case verifies that the system can successfully generate and transmit a patient registration message that conforms to the OpenHIE specification, and that a mock receiving system correctly interprets it.',
  'preconditions': ["The system's interoperability module is running.",
   'A mock OpenHIE-compliant endpoint is configured and available to receive data.',
   "Test patient 'John Doe' does not exist in the system or the mock endpoint."],
  'steps': ["1. Initiate a 'Create Patient' action within the system with valid patient demographic data.",
   '2. Trigger the data exchange process to send the new patient record to the configured OpenHIE endpoint.',
   "3. Monitor the system's outgoing logs for a success confirmation.",
   '4. Query the mock OpenHIE endpoint to confirm the receipt and correct structure of 

# Layer 6 – Coverage Validation Layer

### Create a BigQuery view for reporting — joining requirements ↔ test_cases into a traceability matrix.

- Check coverage: every requirement must have ≥ 1 Positive, 1 Negative, 1 Edge test case.
- Build a traceability matrix mapping:
- requirements → use_cases → test_cases.
- Validate requirement IDs & test case IDs with regex.

In [52]:
from google.cloud import bigquery

project_id = project_id
dataset_id = "requirements_dataset"
requirements_table = "requirements"
testcases_table = "test_cases"
view_name = "requirements_testcases_view"

client = bigquery.Client(project=project_id)

# SQL for the join view
sql = f"""
CREATE OR REPLACE VIEW `{project_id}.{dataset_id}.{view_name}` AS
SELECT 
  r.requirement_id,
  r.title AS requirement_title,
  r.statement AS requirement_statement,
  r.priority AS requirement_priority,
  r.severity AS requirement_severity,
  t.test_id,
  t.title AS test_title,
  t.description AS test_description,
  t.priority AS test_priority,
  t.severity AS test_severity,
  t.execution_status,
  t.owner,
  t.created_at
FROM `{project_id}.{dataset_id}.{requirements_table}` r
LEFT JOIN `{project_id}.{dataset_id}.{testcases_table}` t
ON r.requirement_id = t.requirement_id
ORDER BY r.requirement_id, t.test_id;
"""

# Create or replace the view
job = client.query(sql)
job.result()

print(f"✅ Created or updated view: {project_id}.{dataset_id}.{view_name}")

query = f"""
    SELECT *
    FROM `{project_id}.{dataset_id}.requirements_testcases_view`
    LIMIT 20
"""
results = client.query(query).result()

# Convert to DataFrame
df = results.to_dataframe()



✅ Created or updated view: second-sandbox-470608-m2.requirements_dataset.requirements_testcases_view


In [53]:
df.head(10)

,requirement_id,requirement_title,requirement_statement,requirement_priority,requirement_severity,test_id,test_title,test_description,test_priority,test_severity,execution_status,owner,created_at
0,REQ-001,Stakeholder and Standards Alignment,The system must align with requirements used b...,P1,Major,TC_INTEROP_001,[Positive] Verify successful data exchange usi...,This test case verifies that the system can su...,P1,Critical,Not Executed,QA Team,2025-08-31 06:16:57.213768+00:00
1,REQ-001,Stakeholder and Standards Alignment,The system must align with requirements used b...,P1,Major,TC_INTEROP_002,[Negative] Verify system rejection of a non-co...,This test case ensures that the system correct...,P1,Critical,Not Executed,QA Team,2025-08-31 06:16:57.213792+00:00
2,REQ-001,Stakeholder and Standards Alignment,The system must align with requirements used b...,P1,Major,TC_INTEROP_003,[Edge] Verify handling of a message with a cou...,This test case verifies the system's ability t...,P2,Major,Not Executed,QA Team,2025-08-31 06:16:57.213804+00:00
3,REQ-002,Stakeholder Adoption and Interoperability,The system must be designed to be interoperabl...,P1,Major,None,None,None,None,None,None,None,NaT
4,REQ-003,Adherence to National Health Insurance Systems...,"The system must be designed, developed, and op...",P1,Critical,None,None,None,None,None,None,None,NaT
5,REQ-004,Update National Health Insurance Information S...,A Guide to Common Requirements for National He...,P1,Major,None,None,None,None,None,None,None,NaT
6,REQ-005,Process for Ongoing Requirement Updates,"In addition, this document suggests a way forw...",P2,Major,None,None,None,None,None,None,None,NaT


In [2]:
import importlib
import coverage_validator
importlib.reload(coverage_validator)
from coverage_validator import CoverageValidator

from google.cloud import bigquery
import pandas as pd

# 1. Init validator
cv = CoverageValidator(project_id=project_id, dataset_id=dataset_id)

# 2. Drop the old table (optional, keep if schema changes often)
client = bigquery.Client(project=project_id)
table_ref = f"{project_id}.{dataset_id}.traceability_matrix"
# client.delete_table(table_ref, not_found_ok=True)
print(f"🗑️ Dropped old table {table_ref}")

# 3. Re-run coverage validator (auto-creates table with schema + inserts rows)
trace_matrix = cv.build_traceability_matrix()

# 4. Inspect results
df = pd.DataFrame(trace_matrix)
print(df[["requirement_id", "coverage_status", "coverage_percent", "coverage_gaps"]])


ModuleNotFoundError: No module named 'google'

In [98]:
df

,requirement_id,use_case_id,test_case_ids,coverage_status,coverage_percent,coverage_gaps,compliance,created_at,valid_ids
0,REQ-001,None,"[TC_INTEROP_001, TC_INTEROP_002, TC_INTEROP_003]",FULL,100,[],"[openIMIS, OpenHIE]",2025-08-31T07:32:54.406190,True
1,REQ-002,None,[],NONE,0,"[missing_positive, missing_negative, missing_e...","[openIMIS - Data Model & APIs, openIMIS, OpenH...",2025-08-31T07:32:54.406213,True
2,REQ-003,None,[],NONE,0,"[missing_positive, missing_negative, missing_e...",[National Health Insurance Information Systems...,2025-08-31T07:32:54.406221,True
3,REQ-004,None,[],NONE,0,"[missing_positive, missing_negative, missing_e...",[HIPAA],2025-08-31T07:32:54.406228,True
4,REQ-005,None,[],NONE,0,"[missing_positive, missing_negative, missing_e...","[HIPAA, ISO 9001]",2025-08-31T07:32:54.406234,True


# (Optional) LLM-based semantic validation that the test case really matches the requirement.

In [99]:
import importlib
import semantic_validator
importlib.reload(semantic_validator)
from semantic_validator import SemanticValidator
from google.cloud import bigquery
import pandas as pd

project_id = "second-sandbox-470608-m2"
dataset_id = "requirements_dataset"
requirements_table = "requirements"
testcases_table = "test_cases"

client = bigquery.Client(project=project_id)

# Load requirements & test cases
reqs = [dict(row) for row in client.query(
    f"SELECT distinct requirement_id, statement FROM `{project_id}.{dataset_id}.{requirements_table}`"
).result()]

test_cases = [dict(row) for row in client.query(
    f"SELECT distinct test_id, requirement_id, title, description, steps, expected_result FROM `{project_id}.{dataset_id}.{testcases_table}`"
).result()]

print(f"📥 Loaded {len(reqs)} requirements and {len(test_cases)} test cases")

# Run semantic validation
sv = SemanticValidator(project_id=project_id, dataset_id=dataset_id, model="gemini-2.5-pro")
validated = sv.validate(reqs, test_cases)

# Export results to BigQuery
sv.export_to_bq(validated, table_id="semantic_validation")

# Inspect
df = pd.DataFrame(validated)
print(df[["test_id", "requirement_id", "semantic_matches", "semantic_confidence", "semantic_reason"]])


📥 Loaded 5 requirements and 6 test cases


Semantic validating test cases: 100%|██████████| 6/6 [00:55<00:00,  9.19s/it]


✅ Table second-sandbox-470608-m2.requirements_dataset.semantic_validation exists.
❌ Errors inserting rows: [{'index': 0, 'errors': [{'reason': 'invalid', 'location': 'validation_id', 'debugInfo': '', 'message': 'no such field: validation_id.'}]}, {'index': 1, 'errors': [{'reason': 'invalid', 'location': 'created_at', 'debugInfo': '', 'message': 'no such field: created_at.'}]}, {'index': 2, 'errors': [{'reason': 'invalid', 'location': 'validation_id', 'debugInfo': '', 'message': 'no such field: validation_id.'}]}, {'index': 3, 'errors': [{'reason': 'invalid', 'location': 'validation_id', 'debugInfo': '', 'message': 'no such field: validation_id.'}]}, {'index': 4, 'errors': [{'reason': 'invalid', 'location': 'created_at', 'debugInfo': '', 'message': 'no such field: created_at.'}]}, {'index': 5, 'errors': [{'reason': 'invalid', 'location': 'created_at', 'debugInfo': '', 'message': 'no such field: created_at.'}]}]
          test_id requirement_id  semantic_matches  semantic_confidence  \
0

In [101]:
df.head()

,validation_id,test_id,requirement_id,semantic_matches,semantic_confidence,semantic_reason,created_at
0,8fbf213b-e91a-4c13-969f-af73457b05d4,TC_INTEROP_001,REQ-001,True,100,The test case directly validates a core compon...,2025-08-31T07:35:21.347866
1,dbb7190e-1a95-4197-a74d-796339135b1c,TC_INTEROP_002,REQ-001,True,100,The requirement mandates alignment with openIM...,2025-08-31T07:35:30.897308
2,ba6798f8-35e1-40de-bd15-cb483488f459,TC_INTEROP_003,REQ-001,True,100,The test case directly validates the requireme...,2025-08-31T07:35:41.479806
3,ff9f7574-1809-40b8-a7f4-aae026415b9c,TC_INTEROP_001,REQ-001,False,100,"The test case is a placeholder with no title, ...",2025-08-31T07:35:50.120666
4,4d40f38d-8c46-4d7e-8aa6-e92aa9b34cdd,TC_INTEROP_002,REQ-001,False,100,The test case is a placeholder with no steps o...,2025-08-31T07:35:57.648090
